# <p><center style="font-family:newtimeroman;font-size:180%;"> Plant Village Disease Classification </center></p>
### Table of contents:

* [Import Libraries](#1)
* [Read Data](#2)
* [Visualization Of Dataset](#3)
* [Data Augmentation](#4)
* [Train CNN Model](#5)
* [Evaluate The Model](#6)

In [ ]:
import os
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import keras
from keras.callbacks import EarlyStopping,ModelCheckpoint
import tensorflow as tf
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from tqdm import tqdm

**<a id="2"></a>
# <p style="background-image: url(https://i.postimg.cc/K87ByXmr/stage5.jpg);font-family:camtasia;font-size:120%;color:white;text-align:center;border-radius:15px 50px; padding:7px">Read Data </p>
<a class="btn" href="#home">Tabel of Contents</a>

In [ ]:
def data(dataset_path):
    images = []
    labels = []
    for subfolder in tqdm(os.listdir(dataset_path)):
        subfolder_path = os.path.join(dataset_path, subfolder)
        for image_filename in os.listdir(subfolder_path):
            image_path = os.path.join(subfolder_path, image_filename)
            images.append(image_path)
            labels.append(subfolder)
    df = pd.DataFrame({'image': images, 'label': labels})
    return df

In [ ]:
#train
train = data(r'C:/Users/MY PC/Downloads/plantvillage dataset/color')
train.head()

In [ ]:
train.shape

**<a id="3"></a>
# <p style="background-image: url(https://i.postimg.cc/K87ByXmr/stage5.jpg);font-family:camtasia;font-size:120%;color:white;text-align:center;border-radius:15px 50px; padding:7px">Visualization of Dataset </p>
<a class="btn" href="#home">Tabel of Contents</a>

In [ ]:
train.label.value_counts().to_frame()

In [ ]:
plt.style.use('dark_background')
plt.figure(figsize=(30,50))
for n,i in enumerate(np.random.randint(0,len(train),30)):
    plt.subplot(10,5,n+1)
    img = cv2.imread(train.image[i])
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    plt.imshow(img)
    plt.axis('off')
    text = f'{train.label[i]}\n'
    plt.title(text,fontsize=20)

<a id="4"></a>
# <p style="background-image: url(https://i.postimg.cc/K87ByXmr/stage5.jpg);font-family:camtasia;font-size:120%;color:white;text-align:center;border-radius:15px 50px; padding:7px"> Data Augmentation </p>
<a class="btn" href="#home">Tabel of Contents</a>

In [ ]:
x_train, x_test1, y_train, y_test1 = train_test_split(train['image'], train['label'], test_size=0.2, random_state=42, shuffle=True,stratify=train['label'])
x_val, x_test, y_val, y_test = train_test_split(x_test1,y_test1, test_size=0.5, random_state=42,shuffle=True,stratify=y_test1)
df_train = pd.DataFrame({'image': x_train, 'label': y_train})
df_test = pd.DataFrame({'image': x_test, 'label': y_test})
df_val = pd.DataFrame({'image': x_val, 'label': y_val})

In [ ]:
image_size =  (224, 224)
batch_size = 32
datagen = ImageDataGenerator(
    rescale=1./255
)
train_datagen = ImageDataGenerator(
    rescale=1./255,
    #rotation_range=20,
    #width_shift_range=0.2,
    #height_shift_range=0.2,
    #shear_range=0.2,
    #zoom_range=0.2,
    horizontal_flip=True,
    #fill_mode='nearest'
)
train_generator = train_datagen.flow_from_dataframe(
    df_train,
    x_col='image',
    y_col='label',
    target_size=image_size,
    batch_size=batch_size,
    shuffle=True
)
test_generator = datagen.flow_from_dataframe(
    df_test,
    x_col='image',
    y_col='label',
    target_size=image_size,
    batch_size=batch_size,
    shuffle=False
)
val_generator = datagen.flow_from_dataframe(
    df_val,
    x_col='image',
    y_col='label',
    target_size=image_size,
    batch_size=batch_size,
    shuffle=False
)

In [ ]:
class_ = test_generator.class_indices.keys()
class_ = list(class_)

<a id="5"></a>
# <p style="background-image: url(https://i.postimg.cc/K87ByXmr/stage5.jpg);font-family:camtasia;font-size:120%;color:white;text-align:center;border-radius:15px 50px; padding:7px"> Modeling </p>
<a class="btn" href="#home">Tabel of Contents</a>

In [ ]:
model = keras.models.Sequential([
    keras.layers.Conv2D(32, kernel_size= (3,3), activation= 'relu' , input_shape = (224, 224, 3)),
    keras.layers.MaxPooling2D(2,2),

    keras.layers.Conv2D(32, kernel_size= (3,3), activation= 'relu'),
    keras.layers.MaxPooling2D(2,2),

    keras.layers.Conv2D(32, kernel_size= (3,3), activation= 'relu'),
    keras.layers.MaxPooling2D(2,2),

    keras.layers.Conv2D(32, kernel_size= (3,3), activation= 'relu'),
    keras.layers.MaxPooling2D(2,2),

    keras.layers.Conv2D(32, kernel_size= (3,3), activation= 'relu'),
    keras.layers.MaxPooling2D(2,2),

    keras.layers.Conv2D(32, kernel_size= (3,3), activation= 'relu'),
    keras.layers.MaxPooling2D(2,2),

    keras.layers.Flatten(),

    keras.layers.Dense(64, activation='relu'),

    keras.layers.Dense(38, activation='softmax')


])

model.summary()

<a id="6"></a>
# <p style="background-image: url(https://i.postimg.cc/K87ByXmr/stage5.jpg);font-family:camtasia;font-size:120%;color:white;text-align:center;border-radius:15px 50px; padding:7px"> Evaluate The Model  </p>
<a class="btn" href="#home">Tabel of Contents</a>

In [ ]:
checkpoint_cb = ModelCheckpoint("my_keras_model.keras", save_best_only=True)
early_stopping_cb = EarlyStopping(patience=5, restore_best_weights=True)
model.compile(optimizer ='adam', loss='categorical_crossentropy', metrics=['accuracy',keras.metrics.AUC()])
hist=model.fit(train_generator,epochs=6,validation_data=val_generator,callbacks=[checkpoint_cb,early_stopping_cb])

In [ ]:
hist_=pd.DataFrame(hist)
hist_

In [ ]:
plt.figure(figsize=(25,10))
plt.subplot(1,3,1)
plt.plot(hist_['loss'],'b-o', label='Train_loss')
plt.plot(hist_['val_loss'],'r-o', label='Validation_loss')
plt.title('Train_loss and Validation_loss', fontsize=20)
plt.legend()
plt.subplot(1,3,2)
plt.plot(hist_['accuracy'],'b-o', label='Train_Accuracy')
plt.plot(hist_['val_accuracy'],'r-o', label='Validation_Accuracy')
plt.title('Train_Accuracy and Validation_Accuracy', fontsize=20)
plt.legend()
plt.subplot(1,3,3)
plt.plot(hist_.iloc[:,1], 'b-o', label='Train_acu')
plt.plot(hist_.iloc[:,4], 'r-o', label='Validation_acu')
plt.title('Train_Accuracy and Validation_Accuracy', fontsize=20)
plt.legend()
plt.show()

In [ ]:
score, acc, auc = model.evaluate(test_generator)
print('Test Loss =', score)
print('Test Accuracy =', acc)
print('Test AUC =', auc)

In [ ]:
# Get true labels
y_test = test_generator.classes
# Predict probabilities for each class
predictions = model.predict(test_generator)
# Convert predict probabilities to class labels (choose the class with the highest probability)
y_pred = np.argmax(predictions, axis=1)
# Flatten both arrays just in case
y_test = np.ravel(y_test)
y_pred = np.ravel(y_pred)
# Create a dataframe to compare actual vs predicted labels
df = pd.DataFrame({'Actual': y_test, 'Prediction': y_pred})
df.head()

In [ ]:
batch = next(test_generator)
images = batch[0]
plt.figure(figsize=(50,80))
for n in range(32):
    plt.subplot(8,4,n+1)
    plt.imshow(images[n])
    plt.axis('off')
    text = f'Actual: {class_[int(y_test[n])]}\npred : {class_[int(y_pred[n])]}\n'
    plt.title(text,fontsize=25)

In [ ]:
CM = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(15,25))
sns.heatmap(CM,fmt='g', center= True,cbar=False,annot=True,cmap='Set2',xticklabels=class_,yticklabels=class_)
CM

In [ ]:
ClassificationReport = classification_report(y_test, y_pred, target_names=class_)
print("Classification Report:", ClassificationReport)